In [1]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
import random
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)

# Model training

In [29]:
# Select station to train the models
stationCode = 'CI42'
stationPath = './all data murcia/' + stationCode + '.csv'

In [30]:
def convertirComa(x):
    if type(x) == str:
        return x.replace(",", ".")
    else:
        return x
def leerEstacionDatos(path):
    estacionDatas = pd.read_csv(path, encoding='ISO-8859-1', sep=";")
    estacionDatas.columns = ['ESTACION', 'MUNICIPIO', 'PARAJE', 'HORAS', 'FECHA', 'ETO','TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED','VVMED', '-']
    estacionDatas = estacionDatas.drop(columns=['ESTACION', 'MUNICIPIO', 'PARAJE', 'HORAS', '-'])
    estacionDatas = estacionDatas.reset_index().drop(columns='index')
    estacionDatas['FECHA'] = pd.to_datetime(estacionDatas['FECHA'], format="%d/%m/%y")
    estacionDatas.index = estacionDatas['FECHA']
    estacionDatas.drop(columns='FECHA', inplace=True)
    estacionDatas.dropna(inplace=True)
    for i in estacionDatas.columns:
        estacionDatas[i] = pd.to_numeric(estacionDatas[i].apply(lambda x : convertirComa(x)))
    return estacionDatas
estacionDatas = leerEstacionDatos(stationPath)
estacionDatas

,ETO,TMAX,TMIN,HRMAX,HRMIN,RADMED,VVMED
FECHA,,,,,,,
2010-01-01,2.01,14.76,6.69,71.68,37.73,118.56,2.65
2010-01-02,1.16,15.44,5.08,84.20,43.69,98.32,1.14
2010-01-03,0.71,12.99,1.44,94.50,54.72,95.56,0.61
2010-01-04,0.63,11.71,6.70,92.90,78.59,48.95,0.69
2010-01-05,1.12,16.50,7.74,94.10,49.02,89.83,1.14
...,...,...,...,...,...,...,...
2023-12-27,0.51,16.49,-2.90,83.62,30.30,125.88,0.25
2023-12-28,0.60,15.84,-1.31,80.84,27.92,93.10,0.28
2023-12-29,0.68,14.64,2.84,64.92,32.63,55.33,0.27


In [4]:
# Graphic of each variable
fig = go.Figure()
for c in estacionDatas.columns:
    fig.add_trace(go.Scatter(x= estacionDatas.index, y=estacionDatas[c],
                        name=c, mode='lines'))
fig.show()

## Select dates for model training and validation

In [42]:
estacionDatas = estacionDatas[estacionDatas.index <= '2023-06-17']

# The scaler will be used later
StationScaler = StandardScaler()
# The output is not scaled
estacionDatas_scaled = estacionDatas.drop(columns='ETO')
estacionDatas_scaled = pd.DataFrame(StationScaler.fit_transform(estacionDatas_scaled), index=estacionDatas_scaled.index, columns=estacionDatas_scaled.columns)
estacionDatas_scaled

,TMAX,TMIN,HRMAX,HRMIN,RADMED,VVMED
FECHA,,,,,,
2010-01-01,-1.257123,-0.688523,-1.397739,0.131764,-0.969826,1.836221
2010-01-02,-1.166499,-0.956661,-0.224142,0.548305,-1.188442,-0.239157
2010-01-03,-1.493014,-1.562884,0.741357,1.319187,-1.218254,-0.967601
2010-01-04,-1.663602,-0.686858,0.591376,2.987451,-1.721698,-0.857647
2010-01-05,-1.025231,-0.513651,0.703861,0.920817,-1.280145,-0.239157
...,...,...,...,...,...,...
2023-06-13,0.636667,0.790396,0.841656,0.454653,0.455178,-1.077555
2023-06-14,0.904543,0.547241,0.896024,-0.041562,1.298321,-0.623995
2023-06-15,1.321683,0.757087,0.677615,-0.516113,1.662106,-0.885135


In [6]:
# Split train and test set
train = estacionDatas[estacionDatas.index < '2020-01-01']
test = estacionDatas[estacionDatas.index >= '2020-01-01']
print(train)
print(test)

             ETO   TMAX  TMIN  HRMAX  HRMIN  RADMED  VVMED
FECHA                                                     
2010-01-01  2.01  14.76  6.69  71.68  37.73  118.56   2.65
2010-01-02  1.16  15.44  5.08  84.20  43.69   98.32   1.14
2010-01-03  0.71  12.99  1.44  94.50  54.72   95.56   0.61
2010-01-04  0.63  11.71  6.70  92.90  78.59   48.95   0.69
2010-01-05  1.12  16.50  7.74  94.10  49.02   89.83   1.14
...          ...    ...   ...    ...    ...     ...    ...
2019-12-27  0.61  18.10  3.62  95.88  47.51  116.11   0.26
2019-12-28  0.74  16.00  1.58  97.50  53.18  122.42   0.60
2019-12-29  0.62  15.18  4.32  96.48  54.95  118.79   0.36
2019-12-30  0.61  14.17  2.56  97.82  55.22   89.97   0.36
2019-12-31  0.51  11.18  1.76  97.35  72.04   70.30   0.31

[3592 rows x 7 columns]
             ETO   TMAX   TMIN  HRMAX  HRMIN  RADMED  VVMED
FECHA                                                      
2020-01-01  0.57  13.59   2.01  97.95  59.11   94.02   0.30
2020-01-02  0.51   8.71   5.

## Create all possible combinations of input variables

In [7]:
from itertools import combinations

# List of strings
strings = ['T', 'HR', 'RADMED', 'VVMED']

# Get all unique combinations of the strings
all_combinations = set()

# Loop over different combination lengths
for r in range(1, len(strings) + 1):
    # Generate combinations of length r
    combinations_r = combinations(strings, r)
    # Add unique combinations to the set
    all_combinations.update(combinations_r)

# Convert the combinations to lists 
all = [list(combination) for combination in all_combinations]

all_combinations = []

# Print all unique combinations
for combination in all:
    comb = []
    for c in combination:
        if c == 'T':
            comb.append('TMAX')
            comb.append('TMIN')
        elif c == 'HR':
            comb.append('HRMAX')
            comb.append('HRMIN')
        else:
            comb.append(c)
    all_combinations.append(comb)
all_combinations

[['HRMAX', 'HRMIN'],
 ['TMAX', 'TMIN'],
 ['TMAX', 'TMIN', 'RADMED', 'VVMED'],
 ['HRMAX', 'HRMIN', 'RADMED', 'VVMED'],
 ['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', 'VVMED'],
 ['RADMED'],
 ['RADMED', 'VVMED'],
 ['VVMED'],
 ['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED'],
 ['TMAX', 'TMIN', 'HRMAX', 'HRMIN'],
 ['HRMAX', 'HRMIN', 'RADMED'],
 ['HRMAX', 'HRMIN', 'VVMED'],
 ['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'VVMED'],
 ['TMAX', 'TMIN', 'RADMED'],
 ['TMAX', 'TMIN', 'VVMED']]

## SVR

In [10]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.metrics import  mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from scipy.stats import uniform

# define search space
param_vals = {'kernel': ['rbf'],
              'C': [0.01, 0.1, 1, 10, 50, 100, 1000],
              'gamma': [0.1, 1, 10, 20, 50],
              'epsilon': [0.01, 0.1, 1, 10] }
# define score metrics
scoring_metrics = ['neg_mean_absolute_percentage_error', 'neg_mean_absolute_error', 'r2', 'neg_mean_squared_error']

# lists to save the results
all_medidas = []
svrCV5results = []

# iteration for each input combination
for comb in all_combinations:
    print(comb)

    # scale the original values of each combination
    scaler = StandardScaler().fit(estacionDatas[comb])

    X_train = train[comb]
    if len(comb) == 1:
        X_train = np.array(X_train).reshape(-1,1)

    y_train = train['ETO']
    
    # Fit and transform the training features
    X_train_scaled = scaler.transform(X_train)

    # Fit and transform the test features
    X_test_scaled = scaler.transform(test[comb])
    y_test = test['ETO']

    # define the search
    searchSVR = RandomizedSearchCV(estimator=SVR() , param_distributions=param_vals, 
                                    n_jobs=-1, cv=5, verbose=3, n_iter=60,
                                    scoring=scoring_metrics, refit='neg_mean_absolute_error', random_state=123)
    
    # perform the search
    searchSVR.fit(X_train_scaled, y_train)

    # Make predictions from X_test datas
    y_pred = searchSVR.best_estimator_.predict(X_test_scaled) 

    # Save the best estimator
    svrCV5results.append(pd.DataFrame(searchSVR.cv_results_).iloc[searchSVR.best_index_])

    # Save each measurements (stastistical indicadors)
    medidas = []

    medidas.append(str(comb))
    medidas.append(str(searchSVR.best_estimator_))
    medidas.append(searchSVR.best_score_)
    medidas.append(np.corrcoef(y_test, y_pred)[0][1]**2)
    medidas.append(mean_absolute_error(y_true=y_test,y_pred=y_pred))
    medidas.append(mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False))

    all_medidas.append(medidas)

# Final dataframe results
all_medidas_svr = pd.DataFrame(all_medidas, columns=['combination', 'hyperparameters', 'mean_train_mae', 'test_R2', 'test_MAE', 'test_MAPE', 'test_RMSE'])
all_medidas_svr

['HRMAX', 'HRMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\sklearn\base.py:439: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



['RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\sklearn\base.py:439: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'HRMAX', 'HRMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


,combination,hyperparameters,mean_train_mae,test_R2,test_MAE,test_MAPE,test_RMSE
0,"['HRMAX', 'HRMIN']","SVR(C=10, epsilon=0.01, gamma=0.1)",-1.250337,0.313200,1.323475,71.473502,1.660291
1,"['TMAX', 'TMIN']","SVR(C=1, epsilon=0.01, gamma=1)",-0.827697,0.764168,0.778981,41.522940,1.043562
2,"['TMAX', 'TMIN', 'RADMED', 'VVMED']","SVR(C=100, gamma=0.1)",-0.173682,0.989347,0.153902,7.658692,0.212185
3,"['HRMAX', 'HRMIN', 'RADMED', 'VVMED']","SVR(C=10, gamma=0.1)",-0.420001,0.925318,0.409938,16.242705,0.535226
4,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', '...","SVR(C=10, epsilon=0.01, gamma=0.1)",-0.131295,0.990029,0.147293,7.387990,0.207794
5,['RADMED'],"SVR(C=50, gamma=1)",-0.532014,0.904779,0.545618,27.643795,0.696031
6,"['RADMED', 'VVMED']","SVR(C=1, epsilon=0.01, gamma=1)",-0.437790,0.922225,0.416871,16.576051,0.554950
7,['VVMED'],"SVR(C=1, epsilon=0.01, gamma=10)",-1.277180,0.310469,1.378539,48.293809,1.809674
8,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']","SVR(C=10, epsilon=0.01, gamma=0.1)",-0.244328,0.977038,0.261656,12.505836,0.340103
9,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN']","SVR(C=10, gamma=0.1)",-0.619707,0.801367,0.680936,28.845473,0.887321


In [11]:
svrCV5results = pd.DataFrame(svrCV5results)
svrCV5results['Combination'] = all_medidas_svr['combination'].values
svrCV5results = svrCV5results.reset_index().drop(columns=['index', 'params'])
svrCV5results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_gamma,param_epsilon,param_C,split0_test_neg_mean_absolute_percentage_error,split1_test_neg_mean_absolute_percentage_error,...,rank_test_r2,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,Combination
0,0.475807,0.018449,0.231999,0.005394,rbf,0.1,0.01,10,-0.534283,-0.529802,...,8,-2.499638,-2.470771,-2.234901,-2.655780,-2.528468,-2.477911,0.136945,8,"['HRMAX', 'HRMIN']"
1,0.448611,0.006917,0.228414,0.002236,rbf,1.0,0.01,1,-0.301678,-0.311385,...,6,-0.758138,-1.223777,-1.176894,-1.193843,-1.197567,-1.110044,0.176593,6,"['TMAX', 'TMIN']"
2,1.749336,0.078449,0.135575,0.002956,rbf,0.1,0.10,100,-0.060316,-0.059821,...,1,-0.047845,-0.053768,-0.052546,-0.076552,-0.049766,-0.056095,0.010436,1,"['TMAX', 'TMIN', 'RADMED', 'VVMED']"
3,0.505648,0.010689,0.192507,0.005340,rbf,0.1,0.10,10,-0.128014,-0.146632,...,2,-0.234351,-0.347051,-0.293103,-0.356925,-0.326788,-0.311644,0.044388,2,"['HRMAX', 'HRMIN', 'RADMED', 'VVMED']"
4,1.249756,0.015378,0.221239,0.002575,rbf,0.1,0.01,10,-0.053147,-0.047393,...,2,-0.032860,-0.030221,-0.034136,-0.037140,-0.039849,-0.034841,0.003351,2,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', '..."
5,0.507193,0.023350,0.196379,0.003597,rbf,1.0,0.10,50,-0.175470,-0.186422,...,5,-0.342772,-0.435537,-0.456691,-0.579678,-0.528727,-0.468681,0.081267,5,['RADMED']
6,0.457230,0.005483,0.218385,0.002496,rbf,1.0,0.01,1,-0.134589,-0.141926,...,7,-0.266966,-0.338738,-0.317552,-0.430877,-0.365934,-0.344014,0.054203,7,"['RADMED', 'VVMED']"
7,0.440509,0.005022,0.226432,0.005272,rbf,10.0,0.01,1,-0.678786,-0.525603,...,27,-3.043499,-2.433863,-2.728187,-2.571391,-2.774817,-2.710352,0.205532,27,['VVMED']
8,0.760284,0.017835,0.228300,0.003084,rbf,0.1,0.01,10,-0.089411,-0.093128,...,1,-0.075544,-0.093412,-0.119324,-0.134482,-0.124781,-0.109509,0.021753,1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']"
9,0.481383,0.016638,0.203558,0.002708,rbf,0.1,0.10,10,-0.214452,-0.219386,...,1,-0.565766,-0.759310,-0.676037,-0.675987,-0.711999,-0.677820,0.063830,1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN']"


## Random Forest

In [11]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import  mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold
from skopt import BayesSearchCV

# define search space
param_vals = {'max_depth': range(2, 200, 2),
              'max_features': [None, 'sqrt', 'log2'],
              'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
              'n_estimators': range(10, 2000, 10)}

kf = KFold(n_splits = 5, shuffle = True, random_state = 123)

scoring_metrics = ['neg_mean_absolute_percentage_error', 'neg_mean_absolute_error', 'r2', 'neg_mean_squared_error']

all_medidas_rf = []
rfCV5results = []
for comb in all_combinations:
    print(comb)

    X_train = train[comb]
    if len(comb) == 1:
        X_train = np.array(X_train).reshape(-1,1)

    y_train = train['ETO']

    X_test = test[comb]
    y_test = test['ETO']

    # define the search
    searchRF = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=123),  param_distributions=param_vals, 
                             n_jobs=-1, cv=kf, verbose=3, n_iter=60,
                             scoring=scoring_metrics, refit='neg_mean_absolute_error', random_state=123)
    
    # perform the search
    searchRF.fit(X_train, y_train)

    rfCV5results.append(pd.DataFrame(searchRF.cv_results_).iloc[searchRF.best_index_])

    # Make predictions from X_test datas
    y_pred = searchRF.best_estimator_.predict(X_test) 

    medidas = []

    medidas.append(str(comb))
    medidas.append(str(searchRF.best_estimator_))
    medidas.append(searchRF.best_score_)
    medidas.append(np.corrcoef(y_test, y_pred)[0][1]**2)
    medidas.append(mean_absolute_error(y_true=y_test,y_pred=y_pred))
    medidas.append(mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False))

    all_medidas_rf.append(medidas)

all_medidas_rf = pd.DataFrame(all_medidas_rf, columns=['combination', 'hyperparameters', 'mean_train_mae', 'test_R2', 'test_MAE', 'test_MAPE', 'test_RMSE'])
all_medidas_rf

['RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


['TMAX', 'TMIN', 'HRMAX', 'HRMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'VVMED']
Fitting 5 folds for each of 60 candidates,

c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


['TMAX', 'TMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


,combination,hyperparameters,mean_train_mae,test_R2,test_MAE,test_MAPE,test_CVRMSE,test_MSE,test_RMSE,test_NRMSE
0,['RADMED'],RandomForestRegressor(criterion='friedman_mse'...,-0.536308,0.904717,0.548557,29.625433,21.849082,0.474661,0.688957,0.082510
1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN']",RandomForestRegressor(criterion='friedman_mse'...,-0.624734,0.796245,0.700143,32.175415,28.489521,0.807026,0.898346,0.107586
2,"['HRMAX', 'HRMIN', 'RADMED']","RandomForestRegressor(max_depth=10, max_featur...",-0.474392,0.905918,0.488903,23.936935,20.089120,0.401272,0.633460,0.075864
3,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']",RandomForestRegressor(criterion='friedman_mse'...,-0.250722,0.974166,0.302086,15.725946,12.032664,0.143960,0.379420,0.045440
4,"['TMAX', 'TMIN', 'VVMED']","RandomForestRegressor(max_depth=10, max_featur...",-0.488435,0.873903,0.542678,19.869139,23.308550,0.540192,0.734977,0.088021
5,"['HRMAX', 'HRMIN', 'RADMED', 'VVMED']",RandomForestRegressor(criterion='friedman_mse'...,-0.413018,0.922185,0.417478,16.600001,17.240993,0.295557,0.543652,0.065108
6,"['TMAX', 'TMIN']",RandomForestRegressor(criterion='friedman_mse'...,-0.852311,0.765290,0.813884,46.726153,32.295789,1.037072,1.018367,0.121960
7,"['RADMED', 'VVMED']","RandomForestRegressor(max_depth=10, max_featur...",-0.453998,0.916253,0.423743,17.297334,17.885421,0.318065,0.563972,0.067542
8,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', '...","RandomForestRegressor(criterion='poisson', max...",-0.165899,0.985927,0.174656,8.993874,7.275002,0.052624,0.229399,0.027473
9,"['HRMAX', 'HRMIN']",RandomForestRegressor(criterion='friedman_mse'...,-1.267712,0.302972,1.344377,75.244546,52.112685,2.700251,1.643244,0.196796


In [12]:
rfCV5results = pd.DataFrame(rfCV5results)
rfCV5results['Combination'] = all_medidas_rf['combination'].values
rfCV5results = rfCV5results.reset_index().drop(columns=['index', 'params'])
rfCV5results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,param_criterion,split0_test_neg_mean_absolute_percentage_error,split1_test_neg_mean_absolute_percentage_error,...,rank_test_r2,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,Combination
0,1.315370,0.004061,0.044911,0.001020,550,None,4,friedman_mse,-0.229114,-0.218282,...,1,-0.455357,-0.449392,-0.438657,-0.487948,-0.438241,-0.453919,0.018217,1,['RADMED']
1,2.923063,0.024002,0.062814,0.002482,390,sqrt,12,friedman_mse,-0.242211,-0.240469,...,1,-0.669777,-0.646347,-0.629829,-0.671047,-0.656065,-0.654613,0.015393,1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN']"
2,3.639226,0.021837,0.057613,0.000800,410,None,10,squared_error,-0.184476,-0.183361,...,1,-0.368565,-0.372973,-0.342634,-0.401505,-0.358627,-0.368861,0.019366,1,"['HRMAX', 'HRMIN', 'RADMED']"
3,13.811534,0.053223,0.279663,0.010821,1500,sqrt,94,friedman_mse,-0.108319,-0.113075,...,1,-0.104029,-0.121818,-0.119766,-0.113955,-0.103931,-0.112700,0.007572,1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']"
4,3.291347,0.050800,0.059613,0.000800,410,None,10,squared_error,-0.173997,-0.176702,...,1,-0.384003,-0.434307,-0.440850,-0.437817,-0.403523,-0.420100,0.022468,1,"['TMAX', 'TMIN', 'VVMED']"
5,1.270488,0.015870,0.029007,0.000895,140,sqrt,122,friedman_mse,-0.150343,-0.150799,...,6,-0.291530,-0.286461,-0.271051,-0.317773,-0.294587,-0.292280,0.015102,6,"['HRMAX', 'HRMIN', 'RADMED', 'VVMED']"
6,3.009483,0.023393,0.110425,0.003499,1090,log2,6,friedman_mse,-0.370716,-0.377302,...,1,-0.969283,-1.102426,-1.014710,-1.170893,-1.059755,-1.063413,0.069742,1,"['TMAX', 'TMIN']"
7,2.430151,0.017944,0.058213,0.002136,410,None,10,squared_error,-0.163613,-0.158013,...,3,-0.360339,-0.337199,-0.336952,-0.383681,-0.321760,-0.347986,0.021692,3,"['RADMED', 'VVMED']"
8,11.497209,0.094937,0.170639,0.002417,930,sqrt,170,poisson,-0.068151,-0.070998,...,2,-0.043987,-0.053094,-0.054948,-0.051745,-0.047537,-0.050262,0.003974,1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', '..."
9,1.742795,0.012228,0.048411,0.002801,550,None,4,friedman_mse,-0.566233,-0.577560,...,3,-2.468815,-2.445984,-2.270979,-2.337917,-2.341378,-2.373015,0.073683,3,"['HRMAX', 'HRMIN']"


##  MLP

In [61]:
import random
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping 
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

# define search space
param_vals = {    
    "first_layer": range(12, 1024, 12),
    "second_layer": range(4, 256, 4),
    "dropout": [0, 0.2, 0.4, 0.6, 0.8],
    "loss": ["mse", "mae"],
    "optimizer": ["adam", "sgd", "RMSprop"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.01, 0.1],
    "activation": ["relu", "tanh"],
    "batch_size": [32, 64, 128]
    }

def get_model(first_layer, second_layer, dropout, activation, meta):

    model = Sequential() 
    model.add(Dense(first_layer, activation = activation, input_shape = (X_train_scaled.shape[1], ))) 
    model.add(Dropout(dropout))
    model.add(Dense(second_layer, activation = activation)) 
    model.add(Dense(1))
    return model

scoring_metrics = ['neg_mean_absolute_percentage_error', 'neg_mean_absolute_error', 'r2', 'neg_mean_squared_error']

all_medidas = []
mlpCV5results = []
for comb in all_combinations:

    print(comb)
    scaler = StandardScaler().fit(estacionDatas[comb])

    X_train = train[comb]
    if len(comb) == 1:
        X_train = np.array(X_train).reshape(-1,1)

    y_train = train['ETO']
    
    # Fit and transform the training features
    X_train_scaled = scaler.transform(X_train)

    # Fit and transform the test features
    X_test_scaled = scaler.transform(test[comb])
    y_test = test['ETO']

    modelMLP = KerasRegressor(
        get_model,
        metrics = ['mean_absolute_error'],
        loss=None,
        first_layer = None,
        second_layer= None,
        dropout = None,
        activation=None,
        batch_size=None, 
        optimizer=None,
        epochs = 500, 
        verbose = 0, 
        validation_split = 0.2, 
        callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)],
        random_state=123
    )

    # define the search
    searchMLP = RandomizedSearchCV(estimator=modelMLP, param_distributions=param_vals, 
                                    n_jobs=-1, cv=5, verbose=3, n_iter=60,
                                    scoring=scoring_metrics, refit='neg_mean_absolute_error', random_state=123)
    
    # perform the search
    searchMLP.fit(X_train_scaled, y_train)

    # Make predictions from X_test datas
    y_pred = searchMLP.best_estimator_.predict(X_test_scaled)

    mlpCV5results.append(pd.DataFrame(searchMLP.cv_results_).iloc[searchMLP.best_index_])

    medidas = []

    medidas.append(str(comb))
    medidas.append(str(searchMLP.best_estimator_))
    medidas.append(searchMLP.best_score_)
    medidas.append(np.corrcoef(y_test, y_pred)[0][1]**2)
    medidas.append(mean_absolute_error(y_true=y_test,y_pred=y_pred))
    medidas.append(mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False))

    all_medidas.append(medidas)

all_medidas_mlp = pd.DataFrame(all_medidas, columns=['combination', 'hyperparameters', 'mean_train_mae', 'test_R2', 'test_MAE', 'test_MAPE', 'test_RMSE'])
all_medidas_mlp

['TMAX', 'TMIN']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['HRMAX', 'HRMIN', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits
['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


['RADMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


['RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


...

['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


['HRMAX', 'HRMIN', 'RADMED', 'VVMED']
Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\psyey\Documents\notebooks\notebooks\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,combination,hyperparameters,mean_train_mae,test_R2,test_MAE,test_MAPE,test_CVRMSE,test_MSE,test_RMSE,test_NRMSE
0,"['TMAX', 'TMIN']",KerasRegressor(\n\tmodel=<function get_model a...,-0.423962,0.765457,0.763468,40.423146,31.859714,1.009255,1.004617,0.120313
1,"['HRMAX', 'HRMIN', 'VVMED']",KerasRegressor(\n\tmodel=<function get_model a...,-0.487522,0.550834,1.088050,38.114765,45.280568,2.038642,1.427810,0.170995
2,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']",KerasRegressor(\n\tmodel=<function get_model a...,-0.126109,0.975516,0.302307,14.282772,12.202169,0.148044,0.384765,0.046080
3,['RADMED'],KerasRegressor(\n\tmodel=<function get_model a...,-0.271369,0.903464,0.560478,28.452722,22.639255,0.509614,0.713873,0.085494
4,"['RADMED', 'VVMED']",KerasRegressor(\n\tmodel=<function get_model a...,-0.223355,0.921357,0.417400,17.114504,17.516136,0.305066,0.552328,0.066147
5,"['TMAX', 'TMIN', 'RADMED', 'VVMED']",KerasRegressor(\n\tmodel=<function get_model a...,-0.089312,0.989754,0.156480,7.679355,6.532447,0.042430,0.205984,0.024669
6,"['HRMAX', 'HRMIN']",KerasRegressor(\n\tmodel=<function get_model a...,-0.634899,0.307471,1.328539,73.246170,52.748703,2.766565,1.663299,0.199198
7,"['TMAX', 'TMIN', 'RADMED']",KerasRegressor(\n\tmodel=<function get_model a...,-0.168032,0.975472,0.367081,17.742492,13.978299,0.194279,0.440771,0.052787
8,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'VVMED']",KerasRegressor(\n\tmodel=<function get_model a...,-0.215549,0.866146,0.612008,21.274261,26.412363,0.693636,0.832848,0.099742
9,['VVMED'],KerasRegressor(\n\tmodel=<function get_model a...,-0.647700,0.320162,1.435552,50.141563,58.952010,3.455529,1.858905,0.222623


In [62]:
mlpCV5results = pd.DataFrame(mlpCV5results)
mlpCV5results['Combination'] = all_medidas_mlp['combination'].values
mlpCV5results = mlpCV5results.reset_index().drop(columns=['index', 'params'])
mlpCV5results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_second_layer,param_optimizer__learning_rate,param_optimizer,param_loss,param_first_layer,param_dropout,param_batch_size,param_activation,split0_test_neg_mean_absolute_percentage_error,split1_test_neg_mean_absolute_percentage_error,split2_test_neg_mean_absolute_percentage_error,split3_test_neg_mean_absolute_percentage_error,split4_test_neg_mean_absolute_percentage_error,mean_test_neg_mean_absolute_percentage_error,std_test_neg_mean_absolute_percentage_error,rank_test_neg_mean_absolute_percentage_error,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,split3_test_neg_mean_absolute_error,split4_test_neg_mean_absolute_error,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,Combination
0,31.947804,6.053609,0.355695,0.080968,188,0.0010,RMSprop,mae,684,0.0,128,tanh,-2.618620,-2.214107,-2.032747,-1.585084,-2.022993,-2.094710,0.333898,23,-0.361917,-0.439582,-0.440829,-0.435419,-0.442065,-0.423962,0.031103,1,0.789948,0.671461,0.687026,0.722675,0.694997,0.713221,0.041806,11,-0.200841,-0.309297,-0.309301,-0.302239,-0.304787,-0.285293,0.042313,11,"['TMAX', 'TMIN']"
1,17.848997,3.971453,0.386315,0.068652,144,0.0010,adam,mae,288,0.4,64,relu,-2.231151,-2.295519,-2.026249,-1.176035,-2.093106,-1.964412,0.405637,36,-0.468420,-0.489946,-0.480300,-0.485911,-0.513033,-0.487522,0.014672,1,0.554961,0.476411,0.591995,0.623804,0.580689,0.565572,0.049759,27,-0.425522,-0.492924,-0.403216,-0.409992,-0.419014,-0.430134,0.032304,27,"['HRMAX', 'HRMIN', 'VVMED']"
2,21.475426,10.912275,0.241454,0.079972,216,0.0010,adam,mae,720,0.2,128,tanh,-0.443862,-0.564604,-0.561561,-0.518727,-0.658107,-0.549372,0.069679,6,-0.109072,-0.124926,-0.124336,-0.133058,-0.139151,-0.126109,0.010131,1,0.977683,0.973038,0.970898,0.967522,0.967569,0.971342,0.003798,1,-0.021338,-0.025383,-0.028761,-0.035396,-0.032408,-0.028657,0.004976,1,"['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED']"
3,22.443233,6.578291,0.220756,0.050375,144,0.0010,adam,mae,288,0.4,64,relu,-1.253660,-1.256597,-1.622622,-1.016429,-1.570767,-1.344015,0.224569,17,-0.232276,-0.265205,-0.259456,-0.301097,-0.298809,-0.271369,0.025864,1,0.911695,0.883953,0.883826,0.864537,0.863758,0.881554,0.017468,13,-0.084432,-0.109251,-0.114810,-0.147633,-0.136145,-0.118454,0.021999,13,['RADMED']
4,41.194058,5.613538,0.305268,0.105405,160,0.0100,sgd,mae,168,0.4,32,relu,-1.309900,-1.050919,-1.173498,-0.754967,-1.062429,-1.070343,0.183265,26,-0.196464,-0.225277,-0.216955,-0.239542,-0.238535,-0.223355,0.015876,1,0.927485,0.908478,0.917638,0.910100,0.907156,0.914171,0.007583,1,-0.069335,-0.086162,-0.081396,-0.097977,-0.092779,-0.085530,0.009872,1,"['RADMED', 'VVMED']"
5,18.513760,5.516443,0.416694,0.108170,208,0.0010,adam,mse,312,0.0,128,relu,-0.372629,-0.310263,-0.404032,-0.363258,-0.464620,-0.382960,0.050778,1,-0.082479,-0.083014,-0.088759,-0.103317,-0.088991,-0.089312,0.007522,1,0.986454,0.986783,0.985443,0.984515,0.986952,0.986029,0.000921,1,-0.012952,-0.012443,-0.014386,-0.016876,-0.013038,-0.013939,0.001603,1,"['TMAX', 'TMIN', 'RADMED', 'VVMED']"
6,23.745936,11.438993,0.413493,0.116797,216,0.0010,adam,mae,720,0.2,128,tanh,-2.335472,-2.407454,-2.175308,-1.527043,-2.182097,-2.125475,0.312224,35,-0.609240,-0.631705,-0.610028,-0.657648,-0.665876,-0.634899,0.023513,1,0.335609,0.336400,0.404103,0.365502,0.329019,0.354127,0.027980,28,-0.635254,-0.624735,-0.588903,-0.691499,-0.670505,-0.642179,0.035848,27,"['HRMAX', 'HRMIN']"
7,37.331589,10.831137,0.679353,0.070820,236,0.0001,adam,mae,228,0.0,32

# Train the final models

In [19]:
# Define the best combination of each number of variables
param4 = ['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED', 'VVMED']
param3 = ['TMAX', 'TMIN', 'RADMED', 'VVMED']
param2 = ['TMAX', 'TMIN', 'RADMED']
param1 = ['RADMED']

all_params = [param4, param3, param2, param1]

In [20]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
# Train with all observations of the station
X_train = estacionDatas.drop(columns='ETO')
y_train = estacionDatas['ETO']
X_train_scaled = pd.DataFrame(StationScaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_train_scaled

,TMAX,TMIN,HRMAX,HRMIN,RADMED,VVMED
FECHA,,,,,,
2010-01-01,-1.257123,-0.688523,-1.397739,0.131764,-0.969826,1.836221
2010-01-02,-1.166499,-0.956661,-0.224142,0.548305,-1.188442,-0.239157
2010-01-03,-1.493014,-1.562884,0.741357,1.319187,-1.218254,-0.967601
2010-01-04,-1.663602,-0.686858,0.591376,2.987451,-1.721698,-0.857647
2010-01-05,-1.025231,-0.513651,0.703861,0.920817,-1.280145,-0.239157
...,...,...,...,...,...,...
2023-06-13,0.636667,0.790396,0.841656,0.454653,0.455178,-1.077555
2023-06-14,0.904543,0.547241,0.896024,-0.041562,1.298321,-0.623995
2023-06-15,1.321683,0.757087,0.677615,-0.516113,1.662106,-0.885135


### NOTE: The best models for each station are different, the code will execute the corresponding bloc depending the value of stationCode.

In [ ]:
from sklearn.svm import SVR
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping 
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor

In [45]:
print('Training final models for station: ' + stationCode)
if stationCode == 'CI42':
    ############## BEST MODELS FOR CI42 ##############
    # Model 4 - SVR
    modelo4 = SVR(C=10, epsilon=0.01, gamma=0.1)

    # Model 3 - MLP
    def get_model(first_layer, second_layer, dropout, activation, meta):

        model = Sequential() 
        model.add(Dense(first_layer, activation = activation, input_shape = (X_train_scaled[param3].shape[1], ))) 
        model.add(Dropout(dropout))
        model.add(Dense(second_layer, activation = activation)) 
        model.add(Dense(1))
        return model

    modelo3 =  KerasRegressor(
            get_model,
            metrics = ['mean_absolute_error'],
            loss='mse',
            first_layer = 84,
            second_layer= 164,
            dropout = 0,
            activation='relu',
            batch_size=128, 
            optimizer=Adam(learning_rate=0.001),
            epochs = 500, 
            verbose = 0, 
            validation_split = 0.2, 
            callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)],
            random_state=123
    )

    # Model 2 - SVR
    modelo2 = SVR(C=10, epsilon=0.01, gamma=0.1)

    # Model 1 - SVR
    modelo1 = SVR(C=50, gamma=1)

    # Train all models 
    def entrenarLosModelos(modelos):
        for idx, m in enumerate(modelos):
            m.fit(X_train_scaled[all_params[idx]], estacionDatas['ETO']) 
        return modelos

    modelos = entrenarLosModelos([modelo4, modelo3, modelo2, modelo1])


elif stationCode == 'CA91':

    ############## BEST MODELS FOR CA91 ##############
    # Model 4 - MLP
    def get_model(first_layer, second_layer, dropout, activation, meta):

        model = Sequential() 
        model.add(Dense(first_layer, activation = activation, input_shape = (X_train_scaled[param4].shape[1], ))) 
        model.add(Dropout(dropout))
        model.add(Dense(second_layer, activation = activation)) 
        model.add(Dense(1))
        return model

    modelo4 =  KerasRegressor(
            get_model,
            metrics = ['mean_absolute_error'],
            loss='mae',
            first_layer = 228,
            second_layer= 236,
            dropout = 0,
            activation='relu',
            batch_size=32, 
            optimizer=Adam(learning_rate=0.0001),
            epochs = 500, 
            verbose = 0, 
            validation_split = 0.2, 
            callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)],
            random_state=123
    )

    # Model 3 - SVR
    modelo3 = SVR(C=100, gamma=0.1)

    # Model 2 - SVR
    modelo2 = SVR(C=1000, gamma=0.1)

    # Model 1 - SVR
    modelo1 = SVR(C=1, epsilon=0.01, gamma=10)

    # Train all models 
    def entrenarLosModelos(modelos):
        for idx, m in enumerate(modelos):
            m.fit(X_train_scaled[all_params[idx]], estacionDatas['ETO']) 
        return modelos

    modelos = entrenarLosModelos([modelo4, modelo3, modelo2, modelo1])

elif stationCode == 'CR12':

    ############## BEST MODELS FOR CR12 ##############
    # Modelo 4 - SVR
    modelo4 = SVR(C=10, epsilon=0.01, gamma=0.1)

    # Modelo 3 - SVR
    modelo3 = SVR(C=100, gamma=0.1)

    # Modelo 2 - MLP
    def get_model(first_layer, second_layer, dropout, activation, meta):

        model = Sequential() 
        model.add(Dense(first_layer, activation = activation, input_shape = (X_train_scaled[param2].shape[1], ))) 
        model.add(Dropout(dropout))
        model.add(Dense(second_layer, activation = activation)) 
        model.add(Dense(1))
        return model

    modelo2 =  KerasRegressor(
            get_model,
            metrics = ['mean_absolute_error'],
            loss='mae',
            first_layer = 288,
            second_layer= 144,
            dropout = 0.4,
            activation='relu',
            batch_size=64, 
            optimizer=Adam(learning_rate=0.001),
            epochs = 500, 
            verbose = 0, 
            validation_split = 0.2, 
            callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)],
            random_state=123
    )


    # Modelo 1 - SVR
    modelo1 = SVR(C=1, epsilon=0.01, gamma=1)

    # Entrenar todos los modelos con los datos de la estacion
    def entrenarLosModelos(modelos):
        for idx, m in enumerate(modelos):
            #m.fit(X_train_scaled[all_params[idx]], y_train) # para test local
            m.fit(X_train_scaled[all_params[idx]], estacionDatas['ETO']) 
        return modelos

    modelos = entrenarLosModelos([modelo4, modelo3, modelo2, modelo1])

else:
    print('Error in station code -> ', stationCode, ' no match any of three training stations.')

Training final models for station: CI42


# Estimation test at different scale

## Read stations from other stations of Murcia and SIAR

In [31]:
import os

# Read stations from SIAR
def leerEstacionSiar(path):
       estacion = pd.read_csv(path, encoding="utf-16", sep=';', na_values='0')
       estacion.columns = ['IdProvincia', 'IdEstacion', 'Fecha', 'Año', 'Dia', 'Temp Max (ºC)',
              'Temp Mínima (ºC)', 'Humedad Max (%)', 'Humedad Min (%)',
              'Velviento (m/s)', 'Radiación (MJ/m2)', 'EtPMon']
       estacion = estacion[['Fecha', 'EtPMon', 'Temp Max (ºC)',
              'Temp Mínima (ºC)', 'Humedad Max (%)', 'Humedad Min (%)','Radiación (MJ/m2)', 'Velviento (m/s)'  ]]
       estacion['Fecha'] = pd.to_datetime(estacion['Fecha'], format='%d/%m/%Y')

       estacion.dropna(inplace=True)
       estacion.index = estacion['Fecha']
       estacion.drop(columns='Fecha', inplace=True)
       estacion.columns = estacionDatas.columns
       for i in estacion.columns:
              estacion[i] = pd.to_numeric(estacion[i].apply(lambda x : convertirComa(x)))
       estacion['RADMED'] = estacion['RADMED'].apply(lambda x: x / 0.0864)
       estacion = estacion[estacionDatas.columns]
       estacion = estacion[(estacion.index >= '2017-01-01') & (estacion.index < '2023-01-01')]

       if estacion.duplicated().sum() > 0:
              print(path)
              print('Fechas repetidas:', estacion.duplicated().sum())

       return estacion

# SIAR stations
dirSiar = './siar/'
ficheros = os.listdir(dirSiar)
estacionesSiar = []
nombreEstacionesSiar = []
for f in ficheros:
    if 'csv' not in f:
        continue
    estacionesSiar.append(leerEstacionSiar(dirSiar+f))
    nombreEstacionesSiar.append(f.split('.')[0])

# Stations from Murcia Region
murciaDir = './all data murcia/'
nombreEstacionesMurcia = []
estacionesMurcia = []
for f in os.listdir(murciaDir):
    if stationCode in f:
        continue
    df = leerEstacionDatos(murciaDir+f)
    df = df[df.index >= '2017-01-01']
    df = df[df.index <= '2023-06-17']
    estacionesMurcia.append(df)
    nombreEstacionesMurcia.append(f.split('.')[0])

## Evaluate the models

In [25]:
# Obtain all statistical indicators with y_test and y_pred
def obtenerMedidas(y_test, y_pred, graficas=False):    

    medidas = []

    medidas.append(np.corrcoef(y_test, y_pred)[0][1]**2)
    medidas.append(mean_absolute_error(y_true=y_test,y_pred=y_pred))
    medidas.append(mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False))

    if graficas:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=y_test.index, y=y_test,
                            name='real', mode='lines'))
        fig.add_trace(go.Scatter(x=y_test.index, y=y_pred,
                            name='prediction', mode='lines'))
        fig.show()

        fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'real', 'y':'pred'}, trendline='ols')
        fig.show()
    return medidas

# Evaluate all 4 models 
def evaluarModelos(modelos, estacion, graficas=False):

    medidas_modelos = []

    X_test = estacion.drop(columns='ETO')
    y_test = estacion['ETO']
    X_test_scaled = pd.DataFrame(StationScaler.transform(X_test), index=X_test.index, columns=X_test.columns)

    for idx, modelo in enumerate(modelos):
        y_pred = modelo.predict(X_test_scaled[all_params[idx]]) 
        medidas = obtenerMedidas(y_test, y_pred, graficas)
        medidas.append('modelo'+str(4-idx))
        medidas.append(all_params[idx])

        medidas_modelos.append(medidas)

    return pd.DataFrame(medidas_modelos, columns=['R2', 'MAE', 'MAPE', 'RMSE', 'Modelo', 'Variables'])

# Evaluate a station with all 4 models
def evaluarModelosEstaciones(modelos, estaciones, nombres):
    all_estaciones = []
    for idx, estacion in enumerate(estaciones):
        df = evaluarModelos(modelos, estacion)
        df['Estacion'] = nombres[idx]
        all_estaciones.append(df)

    return pd.concat(all_estaciones)


### Regional (Murcia)

In [32]:
resultados = evaluarModelosEstaciones(modelos, estacionesMurcia, nombreEstacionesMurcia)[['R2', 'MAE', 'MAPE', 'RMSE', 'Modelo', 'Estacion']]
resultados

,R2,MAE,MAPE,RMSE,Modelo,Estacion
0,0.990624,0.135689,4.701121,0.203551,modelo4,AL41
1,0.982075,0.209119,7.311648,0.284180,modelo3,AL41
2,0.928910,0.610582,17.688397,0.794746,modelo2,AL41
3,0.830101,0.764762,21.530040,0.998389,modelo1,AL41
0,0.972949,0.263701,14.118951,0.338757,modelo4,CA73
1,0.949090,0.423641,18.808681,0.522265,modelo3,CA73
2,0.928420,0.603211,26.412813,0.685739,modelo2,CA73
3,0.873681,0.520533,19.904488,0.657250,modelo1,CA73
0,0.984385,0.180555,9.116873,0.240564,modelo4,CA91
1,0.986311,0.162352,8.041021,0.212484,modelo3,CA91


In [27]:
resultados['Modelo'] = resultados['Modelo'].apply(lambda x: x.replace('modelo', 'M'))
resultados.groupby(['Modelo']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]

,R2,MAE,MAPE,RMSE
,mean,mean,mean,mean
Modelo,,,,
M4,0.984518,0.176598,7.678300,0.242965
M3,0.973708,0.244266,9.913994,0.321145
M2,0.921412,0.493673,18.209205,0.642210
M1,0.849853,0.621284,20.907725,0.815600


### National

In [46]:
pd.set_option('display.max_rows', None)

In [47]:
resultados = evaluarModelosEstaciones(modelos, estacionesSiar, nombreEstacionesSiar)[['R2', 'MAE', 'MAPE', 'RMSE', 'Modelo', 'Estacion']]
resultados['Modelo'] = resultados['Modelo'].apply(lambda x: x.replace('modelo', 'M'))
resultados['Estacion'] = resultados['Estacion'].apply(lambda x: x.split(' ')[0])
resultados

,R2,MAE,MAPE,RMSE,Modelo,Estacion
0,0.990289,0.147984,6.369875,0.212792,M4,A19
1,0.978481,0.269700,13.239424,0.358585,M3,A19
2,0.948474,0.522281,17.385844,0.664435,M2,A19
3,0.884681,0.557378,18.560544,0.737200,M1,A19
0,0.985504,0.198406,10.117264,0.290118,M4,AB05
1,0.974678,0.340100,20.167930,0.462442,M3,AB05
2,0.943640,0.555541,19.826414,0.749897,M2,AB05
3,0.834835,0.708317,26.293950,0.956041,M1,AB05
0,0.983299,0.191866,8.686704,0.251333,M4,AL02
1,0.969198,0.316509,12.671124,0.389470,M3,AL02


In [48]:
resultados.groupby(['Modelo']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]
pd.set_option('display.max_rows', 20)

,R2,MAE,MAPE,RMSE
,mean,mean,mean,mean
Modelo,,,,
M4,0.969812,0.221886,11.837402,0.310683
M3,0.949990,0.324301,18.649124,0.422683
M2,0.917075,0.439126,19.619654,0.576196
M1,0.844908,0.591931,26.682398,0.778916


# Forecasting test

## Read real and forecast data

In [63]:
# Function that read the forecast data (hourly) of a specific station and convert it into daily values according each variable:
# T, Hr -> max and min
# U2 and Rs -> mean
def leerPredicciones(path):
    df = pd.read_csv(path)
    df['dates'] = pd.to_datetime(df['dates'])
    df.drop(columns=['Estacion', 'Servicio'], inplace=True)
    # Sacar el DF de WB agrupado por dias y con las variables calculadas 
    punto = [l[1] for l in list(df.groupby([df['dates'].dt.date]))]
    FECHA = []
    TMAX = []
    TMIN = []
    HRMAX = []
    HRMIN = []
    VVMED = []
    RADMED = []
    for p in punto:
        FECHA.append(pd.to_datetime(p['dates']).dt.date.iloc[0])
        TMAX.append(p['temp'].max())
        TMIN.append(p['temp'].min())
        HRMAX.append(p['rh'].max())
        HRMIN.append(p['rh'].min())
        VVMED.append(p['wind'].mean())
        RADMED.append(p['solar_rad'].mean())

    return pd.DataFrame({
        "FECHA": pd.to_datetime(FECHA),
        "TMAX": TMAX,
        "TMIN": TMIN,
        "HRMAX": HRMAX,
        "HRMIN": HRMIN,
        "VVMED": VVMED,
        "RADMED": RADMED
    }
    )

In [64]:
def leerPredictionTest():
    dir = './forecastTest/'
    subdirs = os.listdir(dir)

    dfs_wb = []
    dfs_vc = []
    for subdir in subdirs:

        # Fichero ETo real
        station = subdir.split('-')[0]
        eto = leerEstacionDatos('./all data murcia/' + station + '.csv')
        eto = eto[eto.index >= '2023-06-18']
        eto.reset_index(inplace=True)

        loc = dir+subdir+'/'

        # Ficheros de WB y VC
        df_wb = leerPredicciones(loc + 'WB-'+ subdir + '.csv')
        df_wb = pd.merge(df_wb, eto[['FECHA', 'ETO']], on='FECHA')
        df_wb['VVMED'] = df_wb['VVMED'].apply(lambda x: x*4.87/np.log(67.8*10-5.42))

        df_vc = leerPredicciones(loc + 'VC-'+ subdir + '.csv')
        df_vc = pd.merge(df_vc, eto[['FECHA', 'ETO']], on='FECHA')
        df_vc['VVMED'] = df_vc['VVMED'].apply(lambda x: x*4.87/np.log(67.8*10-5.42))


        dfs_wb.append(df_wb)
        dfs_vc.append(df_vc)

    return dfs_wb, dfs_vc, subdirs

dfs_wb, dfs_vc, locs = leerPredictionTest()

## Test using 2 weather services (WeatherBit and VisualCrossing)

### TEST WB

In [65]:
pd.set_option('display.max_rows', None)
resultadosPredicciones = evaluarModelosEstaciones(modelos=modelos, estaciones=[df[estacionDatas.columns] for df in dfs_wb]  , nombres=locs)[['R2', 'MAE', 'MAPE', 'RMSE', 'Modelo', 'Estacion']]
resultadosPredicciones['Modelo'] = resultadosPredicciones['Modelo'].apply(lambda x: x.replace('modelo', 'M'))
resultadosPredicciones['Estacion'] = resultadosPredicciones['Estacion'].apply(lambda x: x.split('-')[0])
resultadosPredicciones

,R2,MAE,MAPE,RMSE,Modelo,Estacion
0,0.936568,0.474115,29.110778,0.611591,M4,AL41
1,0.932504,0.471643,28.876926,0.618102,M3,AL41
2,0.902612,0.510338,25.743129,0.692278,M2,AL41
3,0.880665,0.665512,29.287841,0.884757,M1,AL41
0,0.931647,0.742169,31.688803,0.840486,M4,CA73
1,0.869370,1.344401,48.385588,1.561664,M3,CA73
2,0.894420,0.999175,37.705130,1.144251,M2,CA73
3,0.901592,0.573526,20.512121,0.726911,M1,CA73
0,0.924308,0.666720,32.139107,0.828546,M4,CA91
1,0.927717,0.791445,34.535949,0.944582,M3,CA91


In [60]:
resultadosPredicciones.groupby(['Modelo']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]

,R2,MAE,MAPE,RMSE
,mean,mean,mean,mean
Modelo,,,,
M4,0.930624,0.567432,22.772131,0.713088
M3,0.916559,0.683020,26.047832,0.845251
M2,0.896832,0.634461,22.884538,0.812786
M1,0.882075,0.608501,21.510553,0.800823


### Test VC

In [67]:
resultadosPredicciones = evaluarModelosEstaciones(modelos=modelos, estaciones=[df[estacionDatas.columns] for df in dfs_vc]  , nombres=locs)[['R2', 'MAE', 'MAPE', 'RMSE', 'Modelo', 'Estacion']]
resultadosPredicciones['Modelo'] = resultadosPredicciones['Modelo'].apply(lambda x: x.replace('modelo', 'M'))
resultadosPredicciones['Estacion'] = resultadosPredicciones['Estacion'].apply(lambda x: x.split('-')[0])
resultadosPredicciones

,R2,MAE,MAPE,RMSE,Modelo,Estacion
0,0.952871,0.449169,30.439053,0.563431,M4,AL41
1,0.946320,0.500548,32.101438,0.612354,M3,AL41
2,0.905847,0.544094,25.719474,0.694706,M2,AL41
3,0.870321,0.638233,29.916521,0.876020,M1,AL41
0,0.868549,0.815895,39.749065,0.952789,M4,CA73
1,0.836708,1.835528,66.922020,2.070904,M3,CA73
2,0.875208,1.177646,48.562678,1.307106,M2,CA73
3,0.889851,0.655219,22.869552,0.818046,M1,CA73
0,0.902962,0.753135,36.751354,0.930406,M4,CA91
1,0.916809,0.955144,40.454299,1.125271,M3,CA91


In [62]:
resultadosPredicciones.groupby(['Modelo']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]

,R2,MAE,MAPE,RMSE
,mean,mean,mean,mean
Modelo,,,,
M4,0.921814,0.586083,24.938282,0.738090
M3,0.909158,0.796088,30.543364,0.960965
M2,0.892022,0.702141,25.779606,0.865820
M1,0.877355,0.619980,22.074400,0.815584
